In [1]:
# BIBLIOTECAS
# =====================
import os, re
from pathlib import Path
import numpy as np
import pandas as pd
from typing import List, Optional
from collections import OrderedDict

In [ ]:
#Setores

CVM_SECTORS = OrderedDict([
    ("Bens Industriais", 1),
    ("Consumo Cíclico", 2),
    ("Consumo Não Cíclico", 3),
    ("Financeiro e Outros", 4),
    ("Materiais Básicos", 5),
    ("Petróleo, Gás e Biocombustíveis", 6),
    ("Saúde", 7),
    ("Tecnologia da Informação", 8),
    ("Telecomunicações", 9),
    ("Utilidade Pública", 10),
])

# -------------------------------
# 2) Mapeamento Ticker -> Setor (CVM, janela 2010-2019)
#    Principais ajustes históricos incluídos
# -------------------------------
TICKER_TO_SECTOR = {
    # Financeiro e Outros
    "BBAS3":"Financeiro e Outros","BBDC3":"Financeiro e Outros","BBDC4":"Financeiro e Outros",
    "BRSR6":"Financeiro e Outros","SANB11":"Financeiro e Outros","ITUB3":"Financeiro e Outros",
    "ITUB4":"Financeiro e Outros","ITSA4":"Financeiro e Outros","BPAC11":"Financeiro e Outros",
    "BIDI11":"Financeiro e Outros","IRBR3":"Financeiro e Outros","BBSE3":"Financeiro e Outros",
    "PSSA3":"Financeiro e Outros","WIZS3":"Financeiro e Outros","BPNM4":"Financeiro e Outros",
    "B3SA3":"Financeiro e Outros","BVMF3":"Financeiro e Outros",
    "BRPR3":"Financeiro e Outros","IGTA3":"Financeiro e Outros","BRML3":"Financeiro e Outros",
    "MULT3":"Financeiro e Outros","ALSC3":"Financeiro e Outros","JHSF3":"Financeiro e Outros",
    "CTIP3":"Financeiro e Outros","SULA11":"Financeiro e Outros","CIEL3":"Financeiro e Outros",
    "BRAP4":"Financeiro e Outros",

    # Bens Industriais (inclui transporte/concessões/aéreas/logística e bens de capital)
    "WEGE3":"Bens Industriais","RAPT4":"Bens Industriais","KEPL3":"Bens Industriais",
    "TUPY3":"Bens Industriais","MYPK3":"Bens Industriais","LEVE3":"Bens Industriais",
    "PLAS3":"Bens Industriais","INPR3":"Bens Industriais","MILS3":"Bens Industriais",
    "MAGG3":"Bens Industriais","EMBR3":"Bens Industriais",
    "CCRO3":"Bens Industriais","ECOR3":"Bens Industriais","ARTR3":"Bens Industriais",
    "RAIL3":"Bens Industriais","RUMO3":"Bens Industriais","ALLL11":"Bens Industriais","ALLL3":"Bens Industriais",
    "RLOG3":"Bens Industriais","GOLL4":"Bens Industriais","AZUL4":"Bens Industriais",
    "LLXL3":"Bens Industriais","PRML3":"Bens Industriais","POMO4":"Bens Industriais","ECOD3":"Bens Industriais",
    "TAMM4":"Bens Industriais",

    # Consumo Cíclico (varejo, moda, locação, educação, viagens/lazer, intermediação imobiliária)
    "LREN3":"Consumo Cíclico","MGLU3":"Consumo Cíclico","VVAR11":"Consumo Cíclico","VVAR3":"Consumo Cíclico",
    "BTOW3":"Consumo Cíclico","LAME3":"Consumo Cíclico","LAME4":"Consumo Cíclico",
    "ARZZ3":"Consumo Cíclico","HGTX3":"Consumo Cíclico","GRND3":"Consumo Cíclico",
    "RENT3":"Consumo Cíclico","LCAM3":"Consumo Cíclico","CVCB3":"Consumo Cíclico",
    "MPLU3":"Consumo Cíclico","SMLS3":"Consumo Cíclico",
    "AEDU3":"Consumo Cíclico","ANIM3":"Consumo Cíclico","ESTC3":"Consumo Cíclico",
    "KROT3":"Consumo Cíclico","SEER3":"Consumo Cíclico","YDUQ3":"Consumo Cíclico",
    "ALPA4":"Consumo Cíclico","BBRK3":"Consumo Cíclico", "BISA3":"Consumo Cíclico", 
    "CYRE3":"Consumo Cíclico", "EVEN3":"Consumo Cíclico", "EZTC3":"Consumo Cíclico", 
    "GFSA3":"Consumo Cíclico", "MRVE3":"Consumo Cíclico", "PDGR3":"Consumo Cíclico", 
    "RSID3":"Consumo Cíclico", "TCSA3":"Consumo Cíclico", "TEND3":"Consumo Cíclico",

    # Consumo Não Cíclico (alimentos/bebidas, higiene/med., agro, varejo alimentar)
    "ABEV3":"Consumo Não Cíclico","AMBV3":"Consumo Não Cíclico","AMBV4":"Consumo Não Cíclico",
    "CRFB3":"Consumo Não Cíclico","PCAR4":"Consumo Não Cíclico","PCAR5":"Consumo Não Cíclico",
    "BRFS3":"Consumo Não Cíclico","JBSS3":"Consumo Não Cíclico","BEEF3":"Consumo Não Cíclico",
    "MDIA3":"Consumo Não Cíclico","NATU3":"Consumo Não Cíclico","RADL3":"Consumo Não Cíclico",
    "SLCE3":"Consumo Não Cíclico","SMTO3":"Consumo Não Cíclico","VAGR3":"Consumo Não Cíclico",
    "CRUZ3":"Consumo Não Cíclico","SMLE3":"Consumo Não Cíclico","HYPE3":"Consumo Não Cíclico",
    "MRFG3":"Consumo Não Cíclico",

    # Materiais Básicos
    "VALE3":"Materiais Básicos","VALE5":"Materiais Básicos",
    "GGBR3":"Materiais Básicos","GGBR4":"Materiais Básicos","GOAU4":"Materiais Básicos",
    "CSNA3":"Materiais Básicos","USIM3":"Materiais Básicos","USIM5":"Materiais Básicos",
    "KLBN11":"Materiais Básicos","KLBN4":"Materiais Básicos","FIBR3":"Materiais Básicos",
    "SUZB3":"Materiais Básicos","SUZB5":"Materiais Básicos","BRKM5":"Materiais Básicos",
    "PMAM3":"Materiais Básicos","MMXM3":"Materiais Básicos","CNFB4":"Materiais Básicos",
    "FFTL4":"Materiais Básicos","CCXC3":"Materiais Básicos","DTEX3":"Materiais Básicos",

    # Petróleo, Gás e Biocombustíveis
    "PETR3":"Petróleo, Gás e Biocombustíveis","PETR4":"Petróleo, Gás e Biocombustíveis",
    "ENAT3":"Petróleo, Gás e Biocombustíveis","QGEP3":"Petróleo, Gás e Biocombustíveis",
    "OGXP3":"Petróleo, Gás e Biocombustíveis","OSXB3":"Petróleo, Gás e Biocombustíveis",
    "UGPA3":"Petróleo, Gás e Biocombustíveis","UGPA4":"Petróleo, Gás e Biocombustíveis",
    "CSAN3":"Petróleo, Gás e Biocombustíveis","RPMG3":"Petróleo, Gás e Biocombustíveis",
    "MPXE3":"Petróleo, Gás e Biocombustíveis","LUPA3":"Petróleo, Gás e Biocombustíveis",
    "ENEV3":"Petróleo, Gás e Biocombustíveis","HRTP3":"Petróleo, Gás e Biocombustíveis","BRDT3":"Petróleo, Gás e Biocombustíveis",

    # Saúde
    "FLRY3":"Saúde","HAPV3":"Saúde","GNDI3":"Saúde","DASA3":"Saúde","QUAL3":"Saúde","AMIL3":"Saúde",
    "ODPV3":"Saúde",

    # Tecnologia da Informação
    "TOTS3":"Tecnologia da Informação","LINX3":"Tecnologia da Informação",
    "POSI3":"Tecnologia da Informação","VLID3":"Tecnologia da Informação",

    # Telecomunicações
    "VIVT4":"Telecomunicações","VIVO4":"Telecomunicações","TIMP3":"Telecomunicações",
    "OIBR3":"Telecomunicações","OIBR4":"Telecomunicações",
    "TCSL3":"Telecomunicações","TCSL4":"Telecomunicações",
    "TNLP3":"Telecomunicações","TNLP4":"Telecomunicações",
    "TLPP4":"Telecomunicações","TMAR5":"Telecomunicações",
    "NETC4":"Telecomunicações","BRTO4":"Telecomunicações",

    # Utilidade Pública
    "ELET3":"Utilidade Pública","ELET6":"Utilidade Pública","CMIG3":"Utilidade Pública",
    "CMIG4":"Utilidade Pública","CPFE3":"Utilidade Pública","CPLE6":"Utilidade Pública",
    "EGIE3":"Utilidade Pública","ENBR3":"Utilidade Pública","ENGI11":"Utilidade Pública",
    "EQTL3":"Utilidade Pública","TAEE11":"Utilidade Pública","TRPL4":"Utilidade Pública",
    "TIET11":"Utilidade Pública","CESP6":"Utilidade Pública","LIGT3":"Utilidade Pública",
    "TBLE3":"Utilidade Pública","SAPR11":"Utilidade Pública","SAPR4":"Utilidade Pública",
    "SBSP3":"Utilidade Pública","GETI4":"Utilidade Pública","TERI3":"Utilidade Pública",
    "ELPL4":"Utilidade Pública","ELPL6":"Utilidade Pública","ALUP11":"Utilidade Pública","CSMG3":"Utilidade Pública",
}

def adicionar_setor_cvm(input_csv: str, output_csv: str = None) -> str:
    """
    Lê um CSV com coluna 'Ticker', adiciona 'SectorID' (CVM) e 'SectorName' e salva.
    - input_csv: caminho de entrada (deve conter 'Ticker')
    - output_csv: caminho de saída; se None, sobrescreve o input

    Também imprime um relatório com mapeados/não mapeados.
    """
    df = pd.read_csv(input_csv)
    if "Ticker" not in df.columns:
        raise ValueError("O arquivo precisa conter a coluna 'Ticker'.")

    # Normaliza ticker
    tickers = df["Ticker"].astype(str).str.strip().str.upper()

    # Aplica mapeamento
    sector_name = tickers.map(TICKER_TO_SECTOR).fillna("Desconhecido")
    sector_id = sector_name.map(lambda s: CVM_SECTORS.get(s, -1))

    # Anexa
    df["SectorID"] = sector_id.values
    df["SectorName"] = sector_name.values

    # Salva
    if output_csv is None:
        output_csv = input_csv
    Path(output_csv).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_csv, index=False, encoding="utf-8")

    # Relatório
    unmapped = df.loc[df["SectorID"] == -1, "Ticker"].unique().tolist()
    print(f"[CVM Setores] Total: {len(df)} | Mapeados: {(df['SectorID']!=-1).sum()} | Não mapeados: {(df['SectorID']==-1).sum()}")
    if unmapped:
        print("[Atenção] Tickers a revisar (legados/raros):", ", ".join(sorted(unmapped)))

    return str(output_csv)


In [ ]:
#Utils
# -----------------------

def to_float_smart(x):
    """
    Converte strings tipo '1.234,56' (BR) ou '1,234.56' (US) e variantes em float.
    Trata negativos e milhares. Retorna NaN se não der.
    """
    import re
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.number)):
        return float(x)

    s = str(x).strip()
    if s in {"", "-", "--", "nan", "NaN", "None", "NULL"}:
        return np.nan

    # mantém apenas dígitos, sinais e separadores
    s = re.sub(r"[^0-9\-\.,]", "", s)

    has_dot   = "." in s
    has_comma = "," in s

    try:
        if has_dot and has_comma:
            # decide pelo separador mais à direita
            if s.rfind(",") > s.rfind("."):
                # BR: 1.234,56 -> 1234.56
                s = s.replace(".", "").replace(",", ".")
            else:
                # US: 1,234.56 -> 1234.56
                s = s.replace(",", "")
            return float(s)

        if has_comma and not has_dot:
            # BR decimal: 1234,56 -> 1234.56
            return float(s.replace(",", "."))

        if has_dot and not has_comma:
            # Pode ser decimal (um ponto) ou milhares (vários pontos)
            if s.count(".") == 1:
                return float(s)  # 1234.56
            else:
                # 109.641.290.194 -> 109641290194
                return float(s.replace(".", ""))

        # Só dígitos e talvez sinal
        return float(s)
    except Exception:
        return np.nan
    
def parse_date_br_any(sr: pd.Series) -> pd.Series:
    """
    Converte a coluna de datas de publicação para datetime,
    assumindo SEMPRE padrão brasileiro (DD/MM/AAAA) quando houver ambiguidade.

    Regras:
    - Se estiver no padrão ISO 'YYYY-MM-DD', usamos isso direto (não é ambíguo).
    - Se estiver no padrão brasileiro 'DD/MM/YYYY', interpretamos como dia/mês/ano.
    - Se vier em qualquer outro formato, tentamos parse com dayfirst=True.
    - No final, retornamos datetime normalizado (sem hora).
    """

    s = sr.astype(str).str.strip()

    # 1) tenta ISO claro: 2024-03-31
    iso_mask = s.str.match(r"^\d{4}-\d{2}-\d{2}$")
    out_iso = pd.to_datetime(
        s.where(iso_mask),
        format="%Y-%m-%d",
        errors="coerce"
    )

    # 2) tenta BR claro: 31/03/2024
    br_mask = s.str.match(r"^\d{2}/\d{2}/\d{4}$")
    out_br = pd.to_datetime(
        s.where(br_mask),
        format="%d/%m/%Y",
        dayfirst=True,
        errors="coerce"
    )

    # 3) começa com ISO e preenche lacunas com BR
    out = out_iso.fillna(out_br)

    # 4) fallback genérico:
    #    qualquer coisa que sobrou a gente interpreta assumindo padrão brasileiro (dayfirst=True)
    still_nat = out.isna()
    if still_nat.any():
        out_fallback = pd.to_datetime(
            s[still_nat],
            errors="coerce",
            dayfirst=True   # <- força semântica brasileira
        )
        out.loc[still_nat] = out_fallback

    # 5) normaliza para "apenas a data" (zera hora)
    out = out.dt.normalize()

    return out


def clean_events(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpeza básica do dataset bruto, antes de padronizar:
    1) Converte AnnounceDate/EventTradeDate para datetime.
    2) Remove linhas sem AnnounceDate.
    3) Dedup por (Ticker, AnnounceDate), mantendo o menor EventTradeDate.
    4) Ordena.
    """
    out = df.copy()

    # normaliza datas vindas em string DD/MM/AAAA
    out["AnnounceDate"] = pd.to_datetime(out["AnnounceDate"], dayfirst=True, errors="coerce")

    if "EventTradeDate" in out.columns:
        out["EventTradeDate"] = pd.to_datetime(out["EventTradeDate"], dayfirst=True, errors="coerce")

    # remove eventos sem data de anúncio válida
    out = out.dropna(subset=["AnnounceDate"]).copy()
    
    if "Year" not in out.columns:
        out["Year"] = out["EventTradeDate"].dt.year

    # ordena pra poder resolver duplicatas determinísticamente
    sort_cols = ["Ticker", "AnnounceDate"]
    if "EventTradeDate" in out.columns:
        sort_cols.append("EventTradeDate")

    out = (
        out.sort_values(sort_cols)
           .drop_duplicates(subset=["Ticker","AnnounceDate"], keep="first")
           .reset_index(drop=True)
    )

    return out

def merge_price_features_into_events(
    events_df: pd.DataFrame,
    prices_dir: str = "dataset/prices_processed",
    features_cols: Optional[List[str]] = None,
    outfile: Optional[str] = None,
) -> pd.DataFrame:
    """
    Faz merge (left) das features vindas de dataset/prices_processed/<TICKER>.SA.csv
    em (Ticker, AnnounceDate). AnnounceDate ≡ Data (pregão T0) no prices.
    Se 'ret_t0_t1' não existir, cria a partir de Daily_Return.shift(-1).
    """
    if features_cols is None:
        features_cols = [
            "ret_t0_t1",
            "MA5","MA50","MA200",
            "RSI9","RSI14","RSI30",
            "MA5_50","MA5_200","MA50_200",
            "MOM_1M","MOM_3M","MOM_6M","MOM_12M",
        ]

    prices_dir = Path(prices_dir)

    ev = events_df.copy()
    # 🔧 Garanta datetime na chave do lado dos eventos
    ev["AnnounceDate"] = parse_date_br_any(ev["AnnounceDate"])

    chunks = []

    for tkr in ev["Ticker"].dropna().unique():
        price_file = prices_dir / f"{tkr}.SA.csv"
        if not price_file.exists():
            alts = list(prices_dir.glob(f"{tkr}*.csv"))
            if not alts:
                continue
            price_file = alts[0]

        px = pd.read_csv(price_file, dtype=str)
        if "Data" not in px.columns:
            continue

        px["Data"] = parse_date_br_any(px["Data"])

        # Cria ret_t0_t1 se necessário
        if "ret_t0_t1" not in px.columns:
            if "Daily_Return" in px.columns:
                dr = pd.to_numeric(px["Daily_Return"].apply(to_float_smart), errors="coerce")
                px["ret_t0_t1"] = dr.shift(-1)
            else:
                px["ret_t0_t1"] = np.nan

        # Garante as colunas pedidas
        for c in features_cols:
            if c not in px.columns:
                px[c] = np.nan

        # Datas de anúncio desse ticker (datetime)
        adates = ev.loc[ev["Ticker"] == tkr, "AnnounceDate"].dropna().unique()
        if adates.size == 0:
            continue

        sub = px.loc[px["Data"].isin(adates), ["Data", *features_cols]].copy()
        if sub.empty:
            continue

        sub["Ticker"] = tkr
        sub = sub.rename(columns={"Data": "AnnounceDate"})

        sub = (
            sub.sort_values(["Ticker","AnnounceDate"])
               .drop_duplicates(["Ticker","AnnounceDate"], keep="first")
        )
        chunks.append(sub)

    if not chunks:
        if outfile:
            Path(os.path.dirname(outfile) or ".").mkdir(parents=True, exist_ok=True)
            ev.to_csv(outfile, index=False)
        return ev

    feat_df = pd.concat(chunks, ignore_index=True)

    merged = ev.merge(
        feat_df,
        on=["Ticker","AnnounceDate"],
        how="left",
        validate="m:1"
    )

    # (opcional) voltar AnnounceDate para DD/MM/AAAA
    merged["AnnounceDate"] = merged["AnnounceDate"].dt.strftime("%d/%m/%Y")

    if outfile:
        Path(os.path.dirname(outfile) or ".").mkdir(parents=True, exist_ok=True)
        merged.to_csv(outfile, index=False)

    return merged

In [16]:
# Caminhos
# -----------------------
RAW_PATH        = "dataset/final/pead_event_dataset_2010_2019.csv"
FINAL_CLEAN_OUT = "dataset/final/final_pead_event_dataset_2010_2019.csv"
os.makedirs(os.path.dirname(FINAL_CLEAN_OUT), exist_ok=True)

In [17]:
# PIPELINE FINAL
# -----------------------

# 1. lê dataset bruto consolidado (saída do EventDatasetBuilder)
raw_df = pd.read_csv(RAW_PATH, dtype=str)

# 2. conversão de tipos básicos antes de limpar
#    - AnnounceDate / EventTradeDate → datetime
#    - CAR_30D, CAR_Sign, Beta etc → numérico
tmp = raw_df.copy()

# converte datas (não vou sobrescrever agora; clean_events faz isso também)
# mas já convertemos numéricos críticos ANTES do winsor
num_like_cols = [
    c for c in tmp.columns
    if c not in ["Ticker","CAR_Sign","AnnounceDate","EventTradeDate","FundSource", "Empresa", "Consolidado", "Convencao", "Moeda", "Data_Demonstracao", "Data_Analise", "QuarterEnd", "Data" ]
]

for c in num_like_cols:
    # tenta converter pra float usando nosso parser robusto
    tmp[c] = tmp[c].apply(to_float_smart)

# 3. limpeza estrutural (remove duplicata, garante coerência temporal)
tmp_clean = clean_events(tmp)

# 4. winsor + z-score
#final_std = winsorize_and_standardize(tmp_clean)

# 5. salva resultado final
#final_std.to_csv(FINAL_CLEAN_OUT, index=False)
tmp_clean.to_csv(FINAL_CLEAN_OUT, index=False)
print(f"Arquivo final salvo em {FINAL_CLEAN_OUT}")

# (Opcional) sanity check rápido
print("Head do final padronizado:")
print(tmp_clean.head())

print("\nDescrição CAR_30D após limpeza (sem padronizar):")
if "CAR_30D" in tmp_clean.columns:
    print(tmp_clean["CAR_30D"].describe(percentiles=[.01,.05,.5,.95,.99]))
else:
    print("Coluna CAR_30D não encontrada em tmp_clean.")

events_path = "dataset/final/pead_event_dataset_2010_2019.csv"
events = pd.read_csv(events_path, dtype=str)

merged = merge_price_features_into_events(
    events_df=events,
    prices_dir="dataset/prices_processed",
    outfile="dataset/final/pead_event_dataset_2010_2019_with_prices.csv"
)
print("OK:", merged.shape, "linhas/colunas")

Arquivo final salvo em dataset/final/final_pead_event_dataset_2010_2019.csv
Head do final padronizado:
  Ticker AnnounceDate EventTradeDate   CAR_30D CAR_Sign      Beta  \
0  ABEV3   2014-03-24     2014-03-25 -0.098386        0  0.694085   
1  ABEV3   2014-05-07     2014-05-08 -0.029948        0  0.588614   
2  ABEV3   2014-07-31     2014-08-01 -0.023337        0  0.610779   
3  ABEV3   2014-10-31     2014-11-03  0.021248        1  0.583170   
4  ABEV3   2015-02-26     2015-02-27  0.035410        1  0.588007   

   EstimationLen    FundSource        Data    Empresa  ...  Outros_PC_MET  \
0           88.0  ABEV3.SA.csv  31/12/2013  AMBEV S/A  ...       4224.941   
1          117.0  ABEV3.SA.csv  31/03/2014  AMBEV S/A  ...       4753.495   
2          175.0  ABEV3.SA.csv  30/06/2014  AMBEV S/A  ...       3437.598   
3          241.0  ABEV3.SA.csv  30/09/2014  AMBEV S/A  ...       3905.387   
4          318.0  ABEV3.SA.csv  31/12/2014  AMBEV S/A  ...       7846.705   

   Outros_PC_Q_Chan

In [19]:
adicionar_setor_cvm("dataset/final/pead_event_dataset_2010_2019_with_prices.csv", "dataset/final/dataprep_pead_event_2010_2019.csv")

[CVM Setores] Total: 4919 | Mapeados: 4919 | Não mapeados: 0


'dataset/final/dataprep_pead_event_2010_2019.csv'